In [1]:
pip install dcor --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install omegaconf --quiet

Note: you may need to restart the kernel to use updated packages.


In [4]:
import json
import math
import os
import pandas as pd
import torch
import time as time
from omegaconf import OmegaConf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
# Read pickles from the path specified in config.dest_file
config_path = "config.yaml"  # Replace with the path to your config file
config = OmegaConf.load(config_path)
df_encoded_path = os.path.join("df_encoded.csv")
# Load the DataFrame from the CSV file
df = pd.read_csv("df_encoded.csv")

In [7]:
df.head()

,surfaceProcessingLocationCavity,hotRunner,hotRunnerCost,manufacturingCost,mouldValidationCost,designTime,hourlyDesignCost,removedChassis,weightChassisProcessed,surfaceProcessingLocationChassis,removedCavity,numberOfCavities,weightCavityProcessed,weightMould,totalTransportationPercentage,percentageAircraft,percentageLorry,percentageTrain,percentageShip,totalDistance,transportCost,injectedMaterial_product,injectedMaterialCost,percentageRecycledMaterial,maxDepth,maxWallThickness,productVolume,materozzaVolume,nAnniProduzione,nProdottiAnno,materialDensity,tolerance,surfaceFinishing,cycleTime,machineCycleTime,maintenanceCost,productionCost,injectedMaterial_materozza,injectionMouldingProcess,memtiEngineValue,steelPrice,runnersType,mouldMaterialName,machineName,EUUSMacchina,CNMacchina,gateDiameter,setupTime,warmupTime,deliveryVolume,deliveryPeriod,mouldDesignCostDisplay,mouldTotalCost,Cost,human health - photochemical oxidation,ecosystem quality - terrestrial ecotoxicity,resources - mineral extraction,resources - non-renewable energy,ecosystem quality - terrestrial acidification & nutrification,resources - total,human health - ionising radiation,human health - respiratory effects (inorganics),human health - total,human health - human toxicity,ecosystem quality - aquatic ecotoxicity,climate change - climate change,human health - ozone layer depletion,ecosystem quality - land occupation,climate change - total,ecosystem quality - total
0,3.0,1.0,40000.0,40000.0,1500.0,40.0,65.0,0.20,6.6725,0.0,0.5,32.0,0.005495,6.677995,100.0,5.0,25.0,40.0,30.0,50.0,4000.0,3.0,1.0,0.0,5.0,1.0,5.0,0.0,1.0,1.0,0.00105,0.0,0.0,360.0,10.0,8000.0,12.0,3.0,2.0,0.0,0.0,1.0,6.0,7.0,4.0,4.0,1.0,15.0,30.0,10.0,2.0,2000.0,38500.0,56093.360588,0.000005,0.000885,0.000019,0.001452,0.000023,0.001471,0.000599,0.002724,0.004244,0.000916,0.000017,0.001938,1.483623e-07,0.000036,0.001938,0.000961
1,3.0,1.0,40000.0,25000.0,3000.0,70.0,50.0,0.15,6.6725,0.0,0.5,16.0,0.005495,6.677995,100.0,5.0,25.0,40.0,30.0,50.0,4000.0,3.0,1.0,0.0,5.0,1.0,5.0,0.0,1.0,1.0,0.00105,0.0,0.0,360.0,10.0,8000.0,12.0,3.0,2.0,0.0,0.0,1.0,6.0,7.0,4.0,4.0,1.0,15.0,30.0,10.0,2.0,2000.0,38500.0,43493.360588,0.000004,0.000805,0.000018,0.001366,0.000022,0.001384,0.000567,0.002586,0.004010,0.000853,0.000016,0.001825,1.379114e-07,0.000032,0.001825,0.000874
2,3.0,0.0,40000.0,40000.0,4500.0,100.0,65.0,0.20,6.6725,0.0,0.3,32.0,0.005495,6.677995,100.0,5.0,25.0,40.0,30.0,50.0,4000.0,3.0,1.0,0.0,5.0,1.0,5.0,0.0,1.0,1.0,0.00105,0.0,0.0,360.0,10.0,8000.0,12.0,3.0,2.0,0.0,0.0,1.0,6.0,7.0,4.0,4.0,1.0,15.0,30.0,10.0,2.0,2000.0,38500.0,102993.360588,0.000005,0.000885,0.000019,0.001452,0.000023,0.001470,0.000599,0.002723,0.004243,0.000916,0.000017,0.001938,1.483223e-07,0.000036,0.001938,0.000960
3,3.0,1.0,10000.0,25000.0,4500.0,70.0,50.0,0.18,6.6725,0.0,0.5,32.0,0.005495,6.677995,100.0,5.0,25.0,40.0,30.0,50.0,4000.0,3.0,1.0,0.0,5.0,1.0,5.0,0.0,1.0,1.0,0.00105,0.0,0.0,360.0,10.0,8000.0,12.0,3.0,2.0,0.0,0.0,1.0,6.0,7.0,4.0,4.0,1.0,15.0,30.0,10.0,2.0,2000.0,38500.0,44993.360588,0.000004,0.000853,0.000018,0.001414,0.000022,0.001432,0.000584,0.002666,0.004146,0.000891,0.000017,0.001891,1.439857e-07,0.000034,0.001891,0.000926
4,3.0,0.0,25000.0,25000.0,3000.0,40.0,65.0,0.18,6.6725,0.0,0.3,32.0,0.005495,6.677995,100.0,5.0,25.0,40.0,30.0,50.0,4000.0,3.0,1.0,0.0,5.0,1.0,5.0,0.0,1.0,1.0,0.00105,0.0,0.0,360.0,10.0,8000.0,12.0,3.0,2.0,0.0,0.0,1.0,6.0,7.0,4.0,4.0,1.0,15.0,30.0,10.0,2.0,2000.0,38500.0,67593.360588,0.000004,0.000852,0.000018,0.001414,0.000022,0.001432,0.000584,0.002666,0.004145,0.000890,0.000017,0.001890,1.439457e-07,0.000034,0.001890,0.000925


In [8]:
df.describe()

,surfaceProcessingLocationCavity,hotRunner,hotRunnerCost,manufacturingCost,mouldValidationCost,designTime,hourlyDesignCost,removedChassis,weightChassisProcessed,surfaceProcessingLocationChassis,removedCavity,numberOfCavities,weightCavityProcessed,weightMould,totalTransportationPercentage,percentageAircraft,percentageLorry,percentageTrain,percentageShip,totalDistance,transportCost,injectedMaterial_product,injectedMaterialCost,percentageRecycledMaterial,maxDepth,maxWallThickness,productVolume,materozzaVolume,nAnniProduzione,nProdottiAnno,materialDensity,tolerance,surfaceFinishing,cycleTime,machineCycleTime,maintenanceCost,productionCost,injectedMaterial_materozza,injectionMouldingProcess,memtiEngineValue,steelPrice,runnersType,mouldMaterialName,machineName,EUUSMacchina,CNMacchina,gateDiameter,setupTime,warmupTime,deliveryVolume,deliveryPeriod,mouldDesignCostDisplay,mouldTotalCost,Cost,human health - photochemical oxidation,ecosystem quality - terrestrial ecotoxicity,resources - mineral extraction,resources - non-renewable energy,ecosystem quality - terrestrial acidification & nutrification,resources - total,human health - ionising radiation,human health - respiratory effects (inorganics),human health - total,human health - human toxicity,ecosystem quality - aquatic ecotoxicity,climate change - climate change,human health - ozone layer depletion,ecosystem quality - land occupation,climate change - total,ecosystem quality - total
count,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.0000,164148.000000,164148.000000,164148.000000,164148.000000,1.641480e+05,164148.0,164148.0,164148.0,164148.0,164148.0,164148.000000,164148.0,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.0,164148.000000,164148.000000,164148.000000,164148.000000,164148.0,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.0,164148.000000,164148.0,164148.000000,164148.000000,164148.00000,164148.000000,164148.0,164148.000000,164148.0,164148.000000,164148.000000,164148.0,164148.0,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,164148.000000,1.641480e+05,164148.000000,164148.000000,164148.000000
mean,3.000000,0.195409,15862.270634,36722.073982,1606.586739,51.724541,50.532934,0.163086,6.6725,0.769793,0.308290,2.740917,0.005495,6.677995e+00,100.0,5.0,25.0,40.0,30.0,1133.808514,4000.0,3.455528,2.275922,0.461194,19.336300,1.319736,9.781782,0.319078,1.0,5381.814874,0.001102,0.147477,0.174690,360.0,14.347540,10185.344933,21.585642,3.501060,2.076979,0.0,0.426347,1.0,5.818901,9.061274,5.43275,4.115469,1.0,41.896764,30.0,6594.545654,3.644126,2000.0,38500.0,66584.271435,0.000040,0.003328,0.000024,0.030269,0.000185,0.030294,0.006191,0.014077,0.023236,0.002927,0.000106,0.019277,1.373095e-06,0.000173,0.019277,0.003792
std,0.554905,0.396516,10592.036270,12479.432960,505.081843,21.184073,2.776695,0.024592,0.0000,1.186966,0.037572,6.279052,0.000000,3.552725e-15,0.0,0.0,0.0,0.0,0.0,2229.091505,0.0,1.700838,2.623105,0.498493,29.477817,0.732969,9.831097,0.466121,0.0,15494.806796,0.000140,0.681254,0.768119,0.0,11.216855,6239.503454,11.208630,1.647026,0.525674,0.0,1.047509,0.0,1.404725,4.282591,3.29176,0.933774,0.0,38.959314,0.0,16912.852510,4.686030,0.0,0.0,15482.078690,0.000101,0.005769,0.000017,0.080793,0.000451,0.080809,0.015016,0.031402,0.051850,0.005354,0.000233,0.048485,3.047302e-06,0.000378,0.048485,0.006825
min,2.000000,0.000000,10000.000000,25000.000000,1500.000000,40.000000,50.000000,0.150000,6.6725,0.000000,0.300000,1.000000,0.005495,6.677995e+00,100.0,5.0,25.0,40.0,30.0,50.000000,4000.0,0.000000,1.000000,0.000000,5.000000,1.000000,5.000000,0.000000,1.0,1.000000,0.000920,0.000000,0.000000,360.0,10.000000,8000.000000,12.000000,0.000000,1.000000,0.0,0.000000,1.0,2.000000,2.000000,0.00000,1.000000,1.0,15.000000,30.

In [9]:
df_encoded = df

In [10]:
feature_column = ['surfaceProcessingLocationCavity', 'hotRunner', 'hotRunnerCost',
       'manufacturingCost', 'mouldValidationCost', 'designTime',
       'hourlyDesignCost', 'removedChassis', 'weightChassisProcessed',
       'surfaceProcessingLocationChassis', 'removedCavity', 'numberOfCavities',
       'weightCavityProcessed', 'weightMould', 'totalTransportationPercentage',
       'percentageAircraft', 'percentageLorry', 'percentageTrain',
       'percentageShip', 'totalDistance', 'transportCost',
       'injectedMaterial_product', 'injectedMaterialCost',
       'percentageRecycledMaterial', 'maxDepth', 'maxWallThickness',
       'productVolume', 'materozzaVolume', 'nAnniProduzione', 'nProdottiAnno',
       'materialDensity', 'tolerance', 'surfaceFinishing', 'cycleTime',
       'machineCycleTime', 'maintenanceCost', 'productionCost',
       'injectedMaterial_materozza', 'injectionMouldingProcess',
       'memtiEngineValue', 'steelPrice', 'runnersType', 'mouldMaterialName',
       'machineName', 'EUUSMacchina', 'CNMacchina', 'gateDiameter',
       'setupTime', 'warmupTime', 'deliveryVolume', 'deliveryPeriod',
       'mouldDesignCostDisplay', 'mouldTotalCost']

target_column = ['Cost',
       'human health - photochemical oxidation',
       'ecosystem quality - terrestrial ecotoxicity',
       'resources - mineral extraction', 'resources - non-renewable energy',
       'ecosystem quality - terrestrial acidification & nutrification',
       'resources - total', 'human health - ionising radiation',
       'human health - respiratory effects (inorganics)',
       'human health - total', 'human health - human toxicity',
       'ecosystem quality - aquatic ecotoxicity',
       'climate change - climate change',
       'human health - ozone layer depletion',
       'ecosystem quality - land occupation', 'climate change - total',
       'ecosystem quality - total']

From the above table, we know hat most of the target variables having distance correlation=1 are redundant or used for deriving the final variables. Hence it is good to eliminate them and keep only four ['human health - total', 'ecosystem quality - total', 'resources - total', 'Cost']

In [11]:
input_variables= ['surfaceProcessingLocationCavity', 'hotRunner',
'hotRunnerCost','manufacturingCost', 'mouldValidationCost',
'designTime','hourlyDesignCost', 'removedChassis', 'weightChassisProcessed',
'surfaceProcessingLocationChassis', 'removedCavity', 'numberOfCavities',
'weightCavityProcessed', 'weightMould',
'totalTransportationPercentage','percentageAircraft', 'percentageLorry', 'percentageTrain','percentageShip', 'totalDistance',
'injectedMaterial_product', 'injectedMaterialCost','percentageRecycledMaterial', 'maxDepth', 'maxWallThickness','productVolume', 'materozzaVolume', 'nAnniProduzione', 'nProdottiAnno',
 'materialDensity', 'tolerance', 'surfaceFinishing', 'cycleTime',
'machineCycleTime',
'maintenanceCost', 'productionCost','transportCost', 'mouldTotalCost',
'injectedMaterial_materozza', 'injectionMouldingProcess','memtiEngineValue', 'steelPrice', 'runnersType', 'mouldMaterialName',
'machineName', 'EUUSMacchina', 'CNMacchina', 'gateDiameter', 'setupTime', 'warmupTime', 'deliveryVolume', 'deliveryPeriod','mouldDesignCostDisplay']

output_variables= ['human health - total', 
                   'ecosystem quality - total',
                    'resources - total', 
                      'Cost']

# Main

In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.gaussian_process.kernels import RBF
from scipy.optimize import Bounds
import dcor
from joblib import Parallel, delayed
import time
from sklearn.gaussian_process import GaussianProcessRegressor
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import numpy as np
from matplotlib import pyplot as pyp
from scipy.optimize import minimize, Bounds, NonlinearConstraint
from joblib import Parallel, delayed
import sklearn.gaussian_process as gpr
from collections import OrderedDict
import torch
import time

# 1. Load your dataframe
df = pd.read_csv(df_encoded_path) # <- replace with actual path
np.random.seed(42)

# 2. Select features and target

selected_features = [
    'surfaceProcessingLocationCavity', 'hotRunner', 'hotRunnerCost',
    'manufacturingCost', 'mouldValidationCost', 'designTime',
    'hourlyDesignCost', 'removedChassis', 'weightChassisProcessed',
    'surfaceProcessingLocationChassis', 'removedCavity', 'numberOfCavities',
    'weightCavityProcessed', 'weightMould', 'totalDistance',
    'injectedMaterial_product', 'injectedMaterialCost',
    'percentageRecycledMaterial', 'maxDepth', 'maxWallThickness',
    'productVolume', 'materozzaVolume', 'nProdottiAnno', 'materialDensity',
    'tolerance', 'surfaceFinishing', 'machineCycleTime', 'maintenanceCost',
    'productionCost', 'injectedMaterial_materozza',
    'injectionMouldingProcess', 'steelPrice', 'mouldMaterialName',
    'machineName', 'EUUSMacchina', 'CNMacchina', 'setupTime',
    'deliveryVolume', 'deliveryPeriod'
]

target_column = ['Cost', 'resources - total', 'human health - total', 'ecosystem quality - total']


# 3. Split the data into features and target and scale them

X_raw = df[selected_features].values
Y_raw = df[target_column].values

scaler_x = MinMaxScaler()
X_scaled = scaler_x.fit_transform(X_raw)

scaler_y = MinMaxScaler()
Y_scaled = scaler_y.fit_transform(Y_raw)

def dcor_filter(X, y, threshold):
    print("Shape of y:", y.shape)  # Add this line for debugging
    selected = []
    for i in range(X.shape[1]):
        corr = np.mean([dcor.distance_correlation(X[:, i], y[:, j]) for j in range(y.shape[1])])
        if corr >= threshold:
            selected.append(i)
    return X[:, selected], selected





# MLP evaluator for multiple targets with threshold maximization + error minimization
def mlp_eval(x_batch, *args):
    X_full, y_full = args
    results = []

    # weights for scalarization
    alpha = 0.5  # weight for threshold reward
    beta = 0.5   # weight for error penalty

    for x in x_batch:
        threshold = x[0]
        hidden_size = int(x[1] * (200 - 10) + 10)

        X_train, X_val, y_train, y_val = train_test_split(
            X_full, y_full, test_size=0.2
        )

        X_sel, selected = dcor_filter(X_train, y_train, threshold)
        if X_sel.shape[1] == 0:
            # no features selected → very bad score
            results.append([1e6])
            continue

        X_val_sel = X_val[:, selected]
        model = MLPRegressor(
            hidden_layer_sizes=(hidden_size,), max_iter=300, random_state=0
        )
        model.fit(X_sel, y_train)
        y_pred = model.predict(X_val_sel)

        error = mean_squared_error(y_val, y_pred)

        # scalarized objective: lower is better
        score = beta * error - alpha * threshold

        print(f"[Eval] Threshold: {threshold:.3f}, Hidden: {hidden_size}, "
              f"Features: {len(selected)}, MSE: {error:.4f}, Score: {score:.4f}")

        results.append([score])

    return results

def ref_model(x_batch):
    """Reference model for VPBO.
    Returns zeros with the same shape as mlp_eval output.
    """

    # Return zeros for each split
    return np.zeros((x_batch.shape[0], splits))


# BO class
class BO:
    def __init__(self, distmod, args, dist_ref, ref_args, dim, bounds,
                 descale, kernel, exp_w, ub, lb, system='default'):
        self.distmod = distmod
        self.args = args
        self.dist_ref = dist_ref
        self.ref_args = ref_args
        self.dim = dim
        self.bounds = bounds
        self.descale = descale
        self.kernel = kernel
        self.exp_w = exp_w
        self.ub = ub
        self.lb = lb
        self.system = system

    def descale(self, x):
        m = (self.ub-self.lb)/(self.bounds.ub-self.bounds.lb)
        b = self.ub-m*self.bounds.ub
        return m*x+b
    
    
    def scale(self, x, use_self = True, lb = None, ub = None):
        if use_self:    
            m = (self.bounds.ub-self.bounds.lb)/(self.ub-self.lb)
            b = self.bounds.ub-m*self.ub
        else:
            m = (self.bounds.ub-self.bounds.lb)/(ub-lb)
            b = self.bounds.ub-m*ub
        return m*x+b

    def optimizer_vpbo(self, trials, split_num, lim_init,
                         f_cores = 1, af_cores = 1, ref_cores = 1, x_init  = None):
        
        # Split partition using variables as split point
        print('Variable Partitioned BO run...')
        start = time.time()
        self.trials_vp = trials
        splits = split_num
        self.time_vp = np.zeros(self.trials_vp)
        self.time_fvp = np.zeros(self.trials_vp)
        div = int(self.dim/splits)        
        ref_mod = self.dist_ref['distrefmod']
        
        x = lim_init*np.ones((splits, self.dim))
        lwr = x.copy()
        upr = x.copy()+1e-6
        for i in range(splits):
            if x_init is None:
                x[i, i*div:(i+1)*div] = np.random.uniform(self.bounds.lb,
                                                          self.bounds.ub,
                                                          (1, div))
            else:
                x_init = x_init.reshape(1, self.dim)
                x[i, i*div:(i+1)*div] = x_init[0, i*div:(i+1)*div]
            lwr[i, i*div:(i+1)*div] = self.bounds.lb[i]
            upr[i, i*div:(i+1)*div] = self.bounds.ub[i]
        x = np.vstack([x, lim_init])
        
        init_pts = int(len(x)/splits)
        splt = int(x.shape[0]/f_cores)
        x_bs = np.array(np.ones(f_cores), dtype = tuple)
        
        if f_cores == 1:
            x_bs[0] = x
        else:
            for i in range(f_cores-1):
                x_bs[i] = x[i*splt:(i+1)*splt, :]
            x_bs[-1] = x[(i+1)*splt:, :]
        
        start_f = time.time()
        y = Parallel(n_jobs = f_cores)(delayed(self.distmod)(self.descale(x_s), *self.args)
                                       for x_s in x_bs)
        if str(type(ref_mod))=="<class '__main__.Network'>":
            y_ref = Parallel(n_jobs = ref_cores,  backend="loky")(delayed(ref_mod)(x_s, *self.ref_args)
                                                 for x_s in torch.from_numpy(x).float())
            y_ref = torch.hstack(y_ref[:]).T.reshape(-1, 1).data.numpy()
            end_f = time.time()
        else:
            y_ref = Parallel(n_jobs = ref_cores,  backend="loky")(delayed(ref_mod)(self.descale(x_s), *self.ref_args)
                                                 for x_s in x_bs)
            y_ref = np.vstack(y_ref[:])
            end_f = time.time()            
        self.time_fvp[0] = end_f-start_f
        for i, yy in enumerate(y):
            print(f"Split {i} shape:", np.array(yy).shape)
 
        y = np.vstack(y[:])
        eps = y-y_ref
        y_bst = np.min(y, axis = 0).reshape(-1, 1).T
        
        bnds_var = {}
        model_vp = {}
        LCB = {}
        
        for i in range(splits):
            model_vp[str(i+1)] = gpr.GaussianProcessRegressor(self.kernel,
                                                              alpha = 1e-6,
                                                              n_restarts_optimizer = 10,
                                                              normalize_y = True)
            model_vp[str(i+1)].fit(x, eps[:, i])
            bnds_var[str(i+1)] = Bounds(lwr[i], upr[i])
            LCB[str(i+1)] = LCB_AF(model_vp[str(i+1)],
                                   self.dim,
                                   self.exp_w,
                                   self.descale,
                                   self.dist_ref['distrefmod'+str(i+1)],
                                   self.ref_args).LCB
        
        restarts = int(round(128/(splits+1), 0))
        x_nxt = x.copy()
        x_nxtbs = np.array(np.ones(f_cores), dtype = tuple)
        
        end = time.time()
        for i in range(init_pts):
            self.time_vp[i] = (i+1)*(end-start)/init_pts
            self.time_fvp[i] = (i+1)*(end_f-start_f)/init_pts
        
        print('ITERATION COUNT IS AT 'f'{init_pts};\
              TOTAL ELAPSED TIME: 'f'{self.time_vp[init_pts-1]:.1f}')
        
        end = time.time()
        self.time_vp[0] = end-start
        
        for i in range(self.trials_vp-init_pts):
            x0 = np.random.uniform(self.bounds.lb,
                                   self.bounds.ub,
                                   (restarts, self.dim))
            
            for j in range(splits):
                opt = Parallel(n_jobs = af_cores,  backend="loky")(delayed(minimize)(LCB[str(j+1)],
                                                                    x_0,
                                                                    method = 'L-BFGS-B',
                                                                    bounds = bnds_var[str(j+1)])
                                                  for x_0 in x0)
                x_nxts = np.array([res.x for res in opt], dtype = 'float')
                funs = np.array([np.atleast_1d(res.fun)[0] for res in opt])
                x_nxt[j] = x_nxts[np.argmin(funs)]
                x_nxt[-1, j*div:(j+1)*div] = x_nxts[np.argmin(funs), j*div:(j+1)*div]
            
            if f_cores == 1:
                x_nxtbs[0] = x_nxt
            else:
                for j in range(f_cores-1):
                    x_nxtbs[j] = x_nxt[j*splt:(j+1)*splt, :]
                x_nxtbs[-1] = x_nxt[(j+1)*splt:, :]
            
            start_f = time.time()
            y_nxt = Parallel(n_jobs = f_cores,  backend="loky")(delayed(self.distmod)(self.descale(x_s), *self.args)
                                             for x_s in x_nxtbs)
            if str(type(ref_mod))=="<class '__main__.Network'>":
                y_ref = Parallel(n_jobs = ref_cores,  backend="loky")(delayed(ref_mod)(x_s, *self.ref_args)
                                                 for x_s in torch.from_numpy(x).float())
                y_ref = torch.hstack(y_ref[:]).T.reshape(-1, 1).data.numpy()
                end_f = time.time()
            else:
                y_ref = Parallel(n_jobs = ref_cores,  backend="loky")(delayed(ref_mod)(self.descale(x_s), *self.ref_args)
                                                 for x_s in x_nxtbs)
                y_ref = np.vstack(y_ref[:])
                end_f = time.time()
            self.time_fvp[i+init_pts] = self.time_fvp[i+init_pts-1]+(end_f-start_f)
            
            x = np.vstack([x, x_nxt])
            y_nxt = np.vstack(y_nxt[:])
            y = np.vstack([y, y_nxt])
            eps_nxt = y_nxt-y_ref
            eps = np.vstack([eps, eps_nxt])
            y_bst = np.vstack([y_bst, np.min(y_nxt, axis = 0).reshape(-1,1).T])
            
            for j in range(splits):
                if any(y_nxt[:, j] < min(y[:, j])):
                    lwr[j] = x_nxt[np.argmin(y_nxt[:, j])]
                    lwr[j, j*div:(j+1)*div] = self.bounds.lb[j]
                    upr[j] = x_nxt[np.argmin(y_nxt[:, j])]+1e-6
                    upr[j, j*div:(j+1)*div] = self.bounds.ub[j]
                
            for j in range(splits):
                model_vp[str(j+1)].fit(x, eps[:, j])
                bnds_var[str(j+1)] = Bounds(lwr[j], upr[j])

            end = time.time()
            self.time_vp[i+init_pts] = end-start
            
            print('ITERATION COUNT IS AT 'f'{init_pts+i+1};\
                  TOTAL ELAPSED TIME: 'f'{self.time_vp[i+init_pts]:.1f}')
            
        self.vpbo_optim = True
        self.model_vp = model_vp
        self.x_vp = self.descale(x)
        self.y_vp = y
        self.y_vpbst = y_bst


    

class LCB_AF():
    def __init__(self, model, dim, exp_w, descale, refmod = None, args = ()):
        self.model = model
        self.dim = dim
        self.exp_w = exp_w
        self.descale = descale
        self.args = args
        
        if refmod:
            self.refmod = refmod
        else:
            def zr(x):
                return 0
            self.refmod = zr
            
    def LCB(self, x):
        x = np.array([x]).reshape(-1,1);
        x = x.reshape(int(x.shape[0]/self.dim), self.dim)
        
        mu, std = self.model.predict(x, return_std=True);
        mu = mu.flatten()
        
        if str(type(self.refmod))=="<class '__main__.Network'>":
            yref = self.refmod(torch.from_numpy(x).float(), *self.args).data.numpy()  
        else:
            yref = self.refmod(self.descale(x), *self.args)
            
        return (yref+mu-self.exp_w*std).flatten()
# Run
'''
bo = BO(
    distmod=mlp_eval,
    args=(X_scaled, Y_scaled),  # MULTI-TARGET support
    dist_ref={'distrefmod': ref_model},
    ref_args=(),
    dim=X_scaled.shape[1],
    bounds=Bounds(np.zeros(X_scaled.shape[1]), np.ones(X_scaled.shape[1])),
    descale=lambda x: x,
    kernel=1.0 * RBF(length_scale=1.0),
    exp_w=0.1,
    ub=np.ones(X_scaled.shape[1]),
    lb=np.zeros(X_scaled.shape[1]),
    system='mlp_opt'
)
'''

"\nbo = BO(\n    distmod=mlp_eval,\n    args=(X_scaled, Y_scaled),  # MULTI-TARGET support\n    dist_ref={'distrefmod': ref_model},\n    ref_args=(),\n    dim=X_scaled.shape[1],\n    bounds=Bounds(np.zeros(X_scaled.shape[1]), np.ones(X_scaled.shape[1])),\n    descale=lambda x: x,\n    kernel=1.0 * RBF(length_scale=1.0),\n    exp_w=0.1,\n    ub=np.ones(X_scaled.shape[1]),\n    lb=np.zeros(X_scaled.shape[1]),\n    system='mlp_opt'\n)\n"

In [ ]:
import numpy as np
from scipy.optimize import Bounds, NonlinearConstraint
import sklearn.gaussian_process as gpr
import time
#import RXTR_SYSTEM
from joblib import Parallel, delayed

import sys
sys.path.append('./../../BO_algos')
import Parallel_Algos as BO_algos

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter('ignore', RuntimeWarning)
warnings.simplefilter('ignore', ConvergenceWarning)

# Define Parameters
exp_w = 2.6
#C0varf = np.loadtxt('C_var_gp.txt')
FR = np.array([0.100, 0.075, 0.075])
R_Frac = 1e-6
ub = np.array([423, 423])
lb = np.array([303, 303])
bounds = Bounds((0, 0), (1, 1))
dim = len(ub)
kernel = gpr.kernels.Matern((1, 1), ((0.06, 5), (0.06, 5)), nu = 2.5)




# Train reference model (GP)
def scale(x, ub = 423, lb = 303, sf = 1):
    m = sf/(ub - lb)
    b = -lb*m
    return m*x+b

TT = np.arange(303, 424, 1)
TT = np.meshgrid(TT, TT)
TT = np.hstack([TT[0].reshape(-1, 1), TT[1].reshape(-1, 1)])

print('Create Statistical Reference Model...')
start = time.time()

Tmod = np.linspace(303, 423, 13)
Tmod = np.meshgrid(Tmod, Tmod)
Tmod = np.hstack([Tmod[0].reshape(-1, 1), Tmod[1].reshape(-1, 1)])
CTREF = np.ones((Tmod.shape[0], 3))

Ctref = Parallel(n_jobs = 5)(delayed(SYST_RECYCLE_REF)(Tmod, FR, R_Frac, Cdist)
                             for Cdist in C0varf)

for i in range(3):
    C = np.vstack(Ctref[:][:]).T[:, i::3]
    C = 8*np.sum(C, axis = 1)
    CTREF[:, i] = C

kergp = gpr.kernels.Matern((5, 5), ((1, 10), (1, 10)), nu = 2.5)
gprefmod = gpr.GaussianProcessRegressor(kernel = kergp,
                                        alpha = 1e-6,
                                        n_restarts_optimizer = 10,
                                        normalize_y = True)
gprefmod.fit(scale(Tmod), CTREF)

end = time.time()
mobdtm = end-start
print(mobdtm)

def SYST_C(T):
    CtR = 0
    for i in range(C0varf.shape[0]):
        Ctr = 8*SYST_RECYCLE(T, FR, R_Frac, C0varf[i])[-1]
        CtR += Ctr
    return CtR

def SYST_C_DIST(T):
    CtR = 0
    for i in range(C0varf.shape[0]):
        Ctr = SYST_RECYCLE(T, FR, R_Frac, C0varf[i])
        Ctr = np.vstack(Ctr[:]).T
        CtR += 8*Ctr
    return CtR

def SYST_C_REFGP(T):
    T = T.flatten()
    T = T.reshape(int(T.shape[0]/2), 2)
    T = scale(T)
    return gprefmod.predict(T)[:, -1]

def SYST_C_DISTGP(T):
    T = T.flatten()
    T = T.reshape(int(T.shape[0]/2), 2)
    T = scale(T)
    return gprefmod.predict(T)

def SYST_C_REF1(T):
    T = T.flatten()
    T = T.reshape(int(T.shape[0]/2), 2)
    T = scale(T)
    return gprefmod.predict(T)[:, 0]

def SYST_C_REF2(T):
    T = T.flatten()
    T = T.reshape(int(T.shape[0]/2), 2)
    T = scale(T)
    return gprefmod.predict(T)[:, 1]

def zr(x):
    x = x.reshape(-1, 1)
    x = x.reshape(int(x.shape[0]/2), 2)
    return np.zeros((x.shape[0], 3))
try:
    # Try to train the reference GP model
    kergp = gpr.kernels.Matern((5, 5), ((1, 10), (1, 10)), nu=2.5)
    gprefmod = gpr.GaussianProcessRegressor(kernel=kergp,
                                            alpha=1e-6,
                                            n_restarts_optimizer=10,
                                            normalize_y=True)
    gprefmod.fit(scale(Tmod), CTREF)

    def SYST_C_REFGP(T):
        T = T.flatten().reshape(int(T.shape[0] / 2), 2)
        T = scale(T)
        return gprefmod.predict(T)[:, -1]

    def SYST_C_DISTGP(T):
        T = T.flatten().reshape(int(T.shape[0] / 2), 2)
        T = scale(T)
        return gprefmod.predict(T)

    def SYST_C_REF1(T):
        T = T.flatten().reshape(int(T.shape[0] / 2), 2)
        T = scale(T)
        return gprefmod.predict(T)[:, 0]

    def SYST_C_REF2(T):
        T = T.flatten().reshape(int(T.shape[0] / 2), 2)
        T = scale(T)
        return gprefmod.predict(T)[:, 1]

except Exception as e:
    print("Reference model not available, using zero functions:", e)

    def SYST_C_REFGP(T):
        return np.zeros((T.shape[0] // 2,))

    def SYST_C_DISTGP(T):
        return np.zeros((T.shape[0] // 2, 3))

    def SYST_C_REF1(T):
        return np.zeros((T.shape[0] // 2,))

    def SYST_C_REF2(T):
        return np.zeros((T.shape[0] // 2,))


# Setup BO class
REACOPTIM = BO_algos.BO(ub = ub,
                        lb = lb,
                        dim = 2,
                        exp_w = exp_w,
                        kernel = kernel,
                        system = SYST_C,
                        bounds = bounds,
                        **{'refmod': SYST_C_REFGP,
                           'distmod': SYST_C_DIST,
                           'ref_distmod': SYST_C_DISTGP,
                           'ref_distmod1': SYST_C_REF1,
                           'ref_distmod2': SYST_C_REF2})


## Generate level sets
gpparts = gpr.GaussianProcessRegressor(kernel = kergp,
                                       alpha = 1e-6,
                                       n_restarts_optimizer = 10,
                                       normalize_y = True)
gpparts.fit(scale(Tmod), (CTREF[:, -1]).reshape(-1, 1))

parts = np.array([-461, -383]) # ref mod with g_1(T_1) and g_2(T_1, T_2)

con1 = lambda x: (gpparts.predict(x.reshape(1, 2))).flatten()
con2 = lambda x: x[0]

nlc1 = NonlinearConstraint(con1, -1e4, parts[0])
nlc21 = NonlinearConstraint(con1, parts[0], parts[1])
nlc22 = NonlinearConstraint(con2, 0.4, 1.1)
nlc31 = NonlinearConstraint(con1, parts[0], parts[1])
nlc32 = NonlinearConstraint(con2, 0, 0.38)
nlc4 = NonlinearConstraint(con1, parts[1], 1e4)

cons = {'1': [nlc1], '2': [nlc21, nlc22], '3': [nlc31, nlc32], '4': [nlc4]}

## VP-BO intial points
lim_init = REACOPTIM.scale(np.array([336, 380]))


# Run_BO
trials_seq = 100
trials_par1 = 25
trials_par2 = 33

x_init = np.linspace(bounds.lb, bounds.ub, 5)
x_init = np.meshgrid(*x_init.T)
x_init = np.reshape(x_init, (dim, -1)).T

MET_VPBO1 = np.zeros((trials_par2, 4))
PARAMS_VPBO1 = np.ones(x_init.shape)
DIST_VPBO1 = np.array([]).reshape(0, dim)
RES_VPBO1 = np.ones((MET_VPBO1.shape[0], x_init.shape[0]))

MET_VPBO2 = np.zeros((trials_par2, 4))
PARAMS_VPBO2 = np.ones(x_init.shape)
DIST_VPBO2 = np.array([]).reshape(0, dim)
RES_VPBO2 = np.ones((MET_VPBO2.shape[0], x_init.shape[0]))




## VP-BO 
for i, x_0 in enumerate(x_init):
    start = time.time()
    REACOPTIM.optimizer_vpbo(trials = trials_par2,
                             split_num = 2,
                             lim_init = lim_init,
                             f_cores = 3,
                             af_cores = 1,
                             ref_cores = 1,
                             x_init  = x_0)
    end = time.time()
    print('Run time '+str(end-start)+'s')
    print('iteration '+str(i+1))
    MET_VPBO1[:, 0] += REACOPTIM.time_vp.flatten()
    MET_VPBO1[:, 1] += REACOPTIM.time_fvp.flatten()
    MET_VPBO1[:, 2] += REACOPTIM.y_vpbst[:, -1].flatten()
    MET_VPBO1[:, 3] += np.min(REACOPTIM.y_vpbst[:, -1])
    print('Best VP-BO value is '+str(np.min(REACOPTIM.y_vpbst[:, -1])))
    PARAMS_VPBO1[i] = REACOPTIM.x_vp[np.argmin(REACOPTIM.y_vp[:, -1])]
    DIST_VPBO1 = np.vstack([DIST_VPBO1, REACOPTIM.x_vp])
    RES_VPBO1[:, i] = REACOPTIM.y_vpbst[:, -1]
MET_VPBO1[:, 0] = MET_VPBO1[:, 0]/(i+1)
MET_VPBO1[:, 1] = MET_VPBO1[:, 1]/(i+1)
MET_VPBO1[:, 2] = MET_VPBO1[:, 2]/(i+1)
MET_VPBO1[:, 3] = MET_VPBO1[:, 3]/(i+1)
REACOPTIM.y_vpbst = MET_VPBO1[:, 2].reshape(-1, 1)
REACOPTIM.time_vp = MET_VPBO1[:, 0].flatten()
REACOPTIM.time_fvp = MET_VPBO1[:, 1].flatten()



# Plot convergence plots
REACOPTIM.plots('R1')
REACOPTIM.plots_time('R1')
REACOPTIM.plot_exptime('R1')

Create Statistical Reference Model...


NameError: name 'C0varf' is not defined

In [ ]:
Tmod = np.linspace(0, 1, 13)
Tmod = np.meshgrid(Tmod, Tmod)
Tmod = np.hstack([Tmod[0].reshape(-1, 1), Tmod[1].reshape(-1, 1)])
CTREF = np.ones((Tmod.shape[0], 3))
print(Tmod.shape)
print(CTREF)

In [19]:
import math
import matplotlib.pyplot as plt
from itertools import product
import time

def mean_relative_error(y_true, y_pred, eps=1e-8):
    return torch.mean(torch.abs((y_pred - y_true) / (y_true.abs() + eps))).item()

def mlp_eval(x_batch, *args):
    X_full, y_full = args
    results = []

    alpha = 0.5  # weight for threshold reward
    beta = 0.5   # weight for error penalty

    for x in x_batch:
        threshold = x[0]
        hidden_size = int(x[1] * (200 - 10) + 10)

        X_train, X_val, y_train, y_val = train_test_split(
            X_full, y_full, test_size=0.2
        )

        X_sel, selected = dcor_filter(X_train, y_train, threshold)
        if X_sel.shape[1] == 0:
            results.append([1e6])
            continue

        X_val_sel = X_val[:, selected]
        model = MLPRegressor(hidden_layer_sizes=(hidden_size,), max_iter=300, random_state=0)
        model.fit(X_sel, y_train)
        y_pred = model.predict(X_val_sel)

        error = mean_relative_error(y_val, y_pred)
        score = beta * error - alpha * threshold  # lower = better

        results.append([score])

    return np.array(results)  # ✅ ensures shape is (n,1)

# ---- GridSearch ----
def run_gridsearch_same_space(X_full, y_full, total_evals, hidden_min=10, hidden_max=200):
    """
    Grid search over same parameter space as BO:
    - threshold ∈ [0,1]
    - hidden_size_scaled ∈ [0,1] (mapped to [hidden_min, hidden_max])
    total_evals ≈ VPBO trials
    """
    start = time.time()
    
    grid_points = int(math.sqrt(total_evals))
    thresholds = np.linspace(0, 1, grid_points)
    hidden_scaled = np.linspace(0, 1, grid_points)
    hidden_sizes = (hidden_scaled * (hidden_max - hidden_min) + hidden_min).astype(int)
    
    best_score = float('inf')
    best_params = None
    eval_count = 0
    
    for th, hs in product(thresholds, hidden_sizes):
        X_sel, selected = dcor_filter(X_full, y_full, th)
        if X_sel.shape[1] == 0:
            continue
        X_train, X_val, y_train, y_val = train_test_split(X_sel, y_full, test_size=0.2)
        model = MLPRegressor(hidden_layer_sizes=(hs,), max_iter=300, random_state=0)
        model.fit(X_train, y_train)
        mse = mean_squared_error(y_val, model.predict(X_val))
        eval_count += 1
        
        if mse < best_score:
            best_score = mse
            best_params = (th, hs)
    
    elapsed = time.time() - start
    return best_score, elapsed, eval_count, best_params


# ---- Serial BO ----
bo_serial = BO(
    distmod=None,                  # not used in sbo
    args=(X_scaled, Y_scaled),
    dist_ref=None,
    ref_args=(),
    dim=X_scaled.shape[1],
    bounds=Bounds(np.zeros(X_scaled.shape[1]), np.ones(X_scaled.shape[1])),
    descale=lambda x: x,
    kernel=1.0 * RBF(length_scale=1.0),
    exp_w=0.1,
    ub=np.ones(X_scaled.shape[1]),
    lb=np.zeros(X_scaled.shape[1]),
    system=mlp_eval                # ✅ callable evaluation function
)

start_sbo = time.time()
bo_serial.optimizer_sbo(trials=30, af_cores=4)
sbo_time = time.time() - start_sbo
sbo_best_mse = np.min(bo_serial.y_sbo)


# ---- VPBO ----
bo_vp = BO(
    distmod=mlp_eval,              # ✅ same evaluation function
    args=(X_scaled, Y_scaled),
    dist_ref={'distrefmod': ref_model},
    ref_args=(),
    dim=X_scaled.shape[1],
    bounds=Bounds(np.zeros(X_scaled.shape[1]), np.ones(X_scaled.shape[1])),
    descale=lambda x: x,
    kernel=1.0 * RBF(length_scale=1.0),
    exp_w=0.1,
    ub=np.ones(X_scaled.shape[1]),
    lb=np.zeros(X_scaled.shape[1]),
    system=None                    # not used in vpbo
)

lim_init = np.random.uniform(0, 1, bo_vp.dim)
x_init = np.random.uniform(0, 1, bo_vp.dim)
start_vp = time.time()
bo_vp.optimizer_vpbo(
    trials=30,
    split_num=4,
    lim_init=lim_init,
    f_cores=4,
    af_cores=2,
    ref_cores=2,
    x_init=x_init
)
vp_time = time.time() - start_vp
vp_best_mse = np.min(bo_vp.y_vp)


# ---- GridSearch Run ----
gs_mse, gs_time, gs_evals, gs_params = run_gridsearch_same_space(
    X_scaled, Y_scaled, total_evals=30
)

# ---- Print summary ----
print(f"[GridSearch] Best MSE={gs_mse:.4f}, Time={gs_time:.2f}s, Evals={gs_evals}, Params={gs_params}")
print(f"[Serial BO] Best MSE={sbo_best_mse:.4f}, Time={sbo_time:.2f}s, Evals={len(bo_serial.y_sbo)}")
print(f"[VPBO]      Best MSE={vp_best_mse:.4f}, Time={vp_time:.2f}s, Evals={len(bo_vp.y_vp)}")


# ---- Plots ----
methods = ["GridSearch", "Serial BO", "VPBO"]
times = [gs_time, sbo_time, vp_time]
mses = [gs_mse, sbo_best_mse, vp_best_mse]
evals = [gs_evals, len(bo_serial.y_sbo), len(bo_vp.y_vp)]

# Accuracy vs Time
fig, ax1 = plt.subplots()
ax1.bar(methods, times, color='skyblue')
ax1.set_ylabel('Time (s)', color='blue')
ax2 = ax1.twinx()
ax2.plot(methods, mses, marker='o', color='red')
ax2.set_ylabel('MSE', color='red')
plt.title('Accuracy vs Time')
plt.tight_layout()
plt.show()

# Cost comparison
plt.bar(methods, evals, color='green')
plt.ylabel('# Function Evaluations')
plt.title('Cost Comparison')
plt.tight_layout()
plt.show()


AttributeError: 'BO' object has no attribute 'optimizer_sbo'

In [34]:
import time
import math
import matplotlib.pyplot as plt
from itertools import product
import time



def mean_relative_error(y_true, y_pred, eps=1e-8):
    return torch.mean(torch.abs((y_pred - y_true) / (y_true.abs() + eps))).item()


def mlp_eval(x_batch, *args):
    X_full, y_full = args
    results = []

    # Ensure x_batch is 2D: (n, d)
    x_batch = np.atleast_2d(x_batch)

    alpha = 0.5  # weight for threshold reward
    beta = 0.5   # weight for error penalty

    for x in x_batch:
        threshold = x[0]
        hidden_size = int(x[1] * (200 - 10) + 10)

        X_train, X_val, y_train, y_val = train_test_split(
            X_full, y_full, test_size=0.2
        )

        X_sel, selected = dcor_filter(X_train, y_train, threshold)
        if X_sel.shape[1] == 0:
            results.append([1e6])
            continue

        X_val_sel = X_val[:, selected]
        model = MLPRegressor(hidden_layer_sizes=(hidden_size,), max_iter=30, random_state=0)
        model.fit(X_sel, y_train)
        y_pred = model.predict(X_val_sel)

        error = mean_relative_error(y_val, y_pred)
        score = beta * error - alpha * threshold  # lower = better
        print(f"[Eval] Threshold: {threshold:.3f}, Hidden: {hidden_size}, Features: {len(selected)}, MSE: {error:.4f}")

        results.append([score])

    return np.array(results)  # always shape (n,1)


# ---- GridSearch ----
def run_gridsearch_same_space(X_full, y_full, total_evals, hidden_min=10, hidden_max=200):
    """
    Grid search over same parameter space as BO:
    - threshold ∈ [0,1]
    - hidden_size_scaled ∈ [0,1] (mapped to [hidden_min, hidden_max])
    total_evals ≈ VPBO trials
    """
    start = time.time()
    
    grid_points = int(math.sqrt(total_evals))
    thresholds = np.linspace(0, 1, grid_points)
    hidden_scaled = np.linspace(0, 1, grid_points)
    hidden_sizes = (hidden_scaled * (hidden_max - hidden_min) + hidden_min).astype(int)
    
    best_score = float('inf')
    best_params = None
    eval_count = 0
    
    for th, hs in product(thresholds, hidden_sizes):
        X_sel, selected = dcor_filter(X_full, y_full, th)
        if X_sel.shape[1] == 0:
            continue
        X_train, X_val, y_train, y_val = train_test_split(X_sel, y_full, test_size=0.2)
        model = MLPRegressor(hidden_layer_sizes=(hs,), max_iter=300, random_state=0)
        model.fit(X_train, y_train)
        mse = mean_squared_error(y_val, model.predict(X_val))
        eval_count += 1
        
        if mse < best_score:
            best_score = mse
            best_params = (th, hs)
    
    elapsed = time.time() - start
    return best_score, elapsed, eval_count, best_params


# ---- VPBO ----
vpbo = BO(
    distmod=mlp_eval,              # ✅ same evaluation function
    args=(X_scaled, Y_scaled),
    dist_ref={'distrefmod': ref_model},
    ref_args=(),
    dim=X_scaled.shape[1],
    bounds=Bounds(np.zeros(X_scaled.shape[1]), np.ones(X_scaled.shape[1])),
    descale=lambda x: x,
    kernel=1.0 * RBF(length_scale=1.0),
    exp_w=0.1,
    ub=np.ones(X_scaled.shape[1]),
    lb=np.zeros(X_scaled.shape[1]),
    system=None                    # not used in vpbo
)

lim_init = np.random.uniform(0, 1, vpbo.dim)
x_init = np.random.uniform(0, 1, vpbo.dim)
div = dim/splits
import tqdm
results = []
for splits in [ 4, 8]:  # different partition counts
    
    start = time.time()
    print(f"Running VPBO with {splits} partitions...")
    vpbo.optimizer_vpbo(trials=10, lim_init=lim_init, split_num= splits, af_cores=splits)  # cores match partitions
    elapsed = time.time() - start
    
    results.append({
        "Partitions": splits,
        "Best_MSE": min(vpbo.y_vpbst),
        "Time_s": elapsed,
        "Evals": len(vpbo.y_vpbst)
    })
    df_res = pd.DataFrame(results)
    df_res.to_csv(f"vpbo_partition_results_{splits}.csv", index=False)



df_results = pd.DataFrame(results)
df_results.to_csv("vpbo_partition_results_30.csv", index=False)

NameError: name 'splits' is not defined

In [30]:
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="multiprocessing.resource_tracker")
results = []
# Run_BO

trials_par1 = 25
trials_par2 = 33

ub = np.array([1, 1])
lb = np.array([0, 0])
bounds = Bounds((0, 0), (1, 1))
dim = len(ub)
kernel = gpr.kernels.Matern((1, 1), ((0.06, 5), (0.06, 5)), nu = 2.5)
def SYST_C_REFGP(T):
    n = max(1, T.shape[0] // 2)
    return np.zeros((n, 1))

def SYST_C_DISTGP(T):
    n = max(1, T.shape[0] // 2)
    return np.zeros((n, 1))

def SYST_C_REF1(T):
    n = max(1, T.shape[0] // 2)
    return np.zeros((n, 1))

def SYST_C_REF2(T):
    n = max(1, T.shape[0] // 2)
    return np.zeros((n, 1))

x_init = np.linspace(bounds.lb, bounds.ub, 2)
x_init = np.meshgrid(*x_init.T)
x_init = np.reshape(x_init, (dim, -1)).T
print(x_init)
MET_VPBO1 = np.zeros((trials_par2, 4))
PARAMS_VPBO1 = np.ones(x_init.shape)
DIST_VPBO1 = np.array([]).reshape(0, dim)
RES_VPBO1 = np.ones((MET_VPBO1.shape[0], x_init.shape[0]))

MET_VPBO2 = np.zeros((trials_par2, 4))
PARAMS_VPBO2 = np.ones(x_init.shape)
DIST_VPBO2 = np.array([]).reshape(0, dim)
RES_VPBO2 = np.ones((MET_VPBO2.shape[0], x_init.shape[0]))
kernel = gpr.kernels.Matern((1, 1), ((0.06, 5), (0.06, 5)), nu = 2.5)

REACOPTIM = BO(
        distmod=mlp_eval,
        args=(X_scaled, Y_scaled),
        dist_ref={  'distrefmod': SYST_C_DISTGP,
                    'distrefmod1': SYST_C_REF1,
                    'distrefmod2': SYST_C_REF1},
        ref_args=(),
        dim=2,
        bounds=Bounds(np.zeros(2), np.ones(2)),
        descale=lambda x: x,
        kernel=kernel,
        exp_w=0.1,
        ub=np.ones(dim),
        lb=np.zeros(dim),
        system=None
    )



lim_init = REACOPTIM.scale(np.array([0, 1]))

## VP-BO 
for i, x_0 in tqdm(enumerate(x_init)):
    start = time.time()
    REACOPTIM.optimizer_vpbo(trials = trials_par2,
                             split_num = 2,
                             lim_init = lim_init,
                             f_cores = 4,
                             af_cores = 2,
                             ref_cores = 2,
                             x_init  = x_0)
    end = time.time()
    print('Run time '+str(end-start)+'s')
    print('iteration '+str(i+1))
    MET_VPBO1[:, 0] += REACOPTIM.time_vp.flatten()
    MET_VPBO1[:, 1] += REACOPTIM.time_fvp.flatten()
    MET_VPBO1[:, 2] += REACOPTIM.y_vpbst[:, -1].flatten()
    MET_VPBO1[:, 3] += np.min(REACOPTIM.y_vpbst[:, -1])
    print('Best VP-BO value is '+str(np.min(REACOPTIM.y_vpbst[:, -1])))
    PARAMS_VPBO1[i] = REACOPTIM.x_vp[np.argmin(REACOPTIM.y_vp[:, -1])]
    DIST_VPBO1 = np.vstack([DIST_VPBO1, REACOPTIM.x_vp])
    RES_VPBO1[:, i] = REACOPTIM.y_vpbst[:, -1]
MET_VPBO1[:, 0] = MET_VPBO1[:, 0]/(i+1)
MET_VPBO1[:, 1] = MET_VPBO1[:, 1]/(i+1)
MET_VPBO1[:, 2] = MET_VPBO1[:, 2]/(i+1)
MET_VPBO1[:, 3] = MET_VPBO1[:, 3]/(i+1)
REACOPTIM.y_vpbst = MET_VPBO1[:, 2].reshape(-1, 1)
REACOPTIM.time_vp = MET_VPBO1[:, 0].flatten()
REACOPTIM.time_fvp = MET_VPBO1[:, 1].flatten()

[[0. 0.]
 [1. 0.]
 [0. 1.]
 [1. 1.]]


0it [00:00, ?it/s]

Variable Partitioned BO run...


Exception ignored in: <function ResourceTracker.__del__ at 0x7f34071aaac0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/multiprocessing/resource_tracker.py", line 77, in __del__
  File "/opt/conda/lib/python3.12/multiprocessing/resource_tracker.py", line 86, in _stop
  File "/opt/conda/lib/python3.12/multiprocessing/resource_tracker.py", line 111, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x7f9062e8aac0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/multiprocessing/resource_tracker.py", line 77, in __del__
  File "/opt/conda/lib/python3.12/multiprocessing/resource_tracker.py", line 86, in _stop
  File "/opt/conda/lib/python3.12/multiprocessing/resource_tracker.py", line 111, in _stop_locked
ChildProcessError: [Errno 10] No child processes


Shape of y: (131318, 4)
[Eval] Threshold: 0.000, Hidden: 200, Features: 39, MSE: 0.0000, Score: 0.0000
Shape of y: (131318, 4)
[Eval] Threshold: 0.000, Hidden: 10, Features: 39, MSE: 0.0001, Score: 0.0001
Shape of y: (131318, 4)
[Eval] Threshold: 0.000, Hidden: 200, Features: 39, MSE: 0.0000, Score: 0.0000


Exception ignored in: <function ResourceTracker.__del__ at 0x7ffa48a7eac0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/multiprocessing/resource_tracker.py", line 77, in __del__
  File "/opt/conda/lib/python3.12/multiprocessing/resource_tracker.py", line 86, in _stop
  File "/opt/conda/lib/python3.12/multiprocessing/resource_tracker.py", line 111, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x7fe745bcaac0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/multiprocessing/resource_tracker.py", line 77, in __del__
  File "/opt/conda/lib/python3.12/multiprocessing/resource_tracker.py", line 86, in _stop
  File "/opt/conda/lib/python3.12/multiprocessing/resource_tracker.py", line 111, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x7f8a2d27aac0>
Traceback (most recent call last):
  File "/opt

Split 0 shape: (0,)
Split 1 shape: (0,)
Split 2 shape: (0,)
Split 3 shape: (3, 1)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 3 has size 1

In [19]:
for trials in [ 30, 50]:  # different trial counts
    splits = 4  # keep partitions constant for simplicity

    
    # Reset VPBO internal state
    vpbo.y_vpbst = []
    
    lim_init = np.random.uniform(0, 1, vpbo.dim)
    x_init = np.random.uniform(0, 1, vpbo.dim)

    
    start = time.time()
    print(f"Running VPBO with {splits} partitions...")
    
    vpbo.optimizer_vpbo(trials=trials, lim_init=lim_init, split_num=splits, af_cores=splits, f_cores=splits, x_init=x_init)
    elapsed = time.time() - start
    
    results.append({
        "Partitions": splits,
        "Best_MSE": min(vpbo.y_vpbst),
        "Time_s": elapsed,
        "Evals": len(vpbo.y_vpbst)
    })

df_results = pd.DataFrame(results)
df_results.to_csv("vpbo_partition_results4_50.csv", index=False)


NameError: name 'vpbo' is not defined

In [36]:
df_results = pd.DataFrame(results)
df_results.to_csv("vpbo_partition_results_30.csv", index=False)


In [12]:

lim_init = np.random.uniform(0, 1, bo.dim)
x_init = np.random.uniform(0, 1, bo.dim)
start_time = time.time()
bo.optimizer_vpbo(
    trials=4,
    split_num=6,
    lim_init=lim_init,
    f_cores=6,  # PARALLEL CPU
    af_cores=2,
    ref_cores=2,
    x_init=x_init
)
end_time = time.time()
# Best result
best_idx = np.argmin(bo.y_vp)
best_input = bo.x_vp[best_idx]
best_error = bo.y_vp[best_idx]

threshold_best = best_input[0]
hidden_size_best = int(best_input[1] * (200 - 10) + 10)

print("\n🎯 Best Hyperparameters Found:")
print(f"Distance Threshold: {threshold_best:.3f}")
print(f"Hidden Layer Size: {hidden_size_best}")
print(f"Validation MSE: {best_error[0]:.4f}")
print(f"Total time taken for optimization: {end_time - start_time:.2f} seconds")

# 🔍 Visualize convergence
plt.plot(bo.y_vpbst[:, 0], marker='o')
plt.xlabel("Iteration")
plt.ylabel("Best MSE so far")
plt.title("Convergence of VPBO")
plt.grid(True)
plt.tight_layout()
plt.show()


# Save results
results = {
    "best_input": best_input.tolist(),
    "best_error": best_error.tolist(),
    "y_vpbst": bo.y_vpbst.tolist(),
    "x_vp": bo.x_vp.tolist()
}

results_path = os.path.join(config.dest_file, "bo_results_6.json")
with open(results_path, 'w') as f:
    json.dump(results, f, indent=4)

Variable Partitioned BO run...


KeyboardInterrupt: 

# CURRENT

In [20]:
from colorama import init


def mean_relative_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8)))


def median_relative_error(y_true, y_pred):
    return np.median(np.abs((y_true - y_pred) / (y_true + 1e-8)))

# ...existing code...
def mlp_eval(x_batch, *args, splits=4):
    """
    Evaluates a batch of hyperparameter candidates for VPBO.
    Each row in x_batch is [threshold, hidden_size_scaled].
    Returns a (n, splits) array: each candidate gets a vector of identical errors for each split.
    """
    X_full, y_full = args
    results = []
    x_batch = np.atleast_2d(x_batch)  # Ensure shape (n, d)
    for x in x_batch:
        threshold = x[0]
        hidden_size = int(x[1] * (200 - 10) + 10)
        X_train, X_val, y_train, y_val = train_test_split(X_full, y_full, test_size=0.2)
        X_sel, selected = dcor_filter(X_train, y_train, threshold)
        if X_sel.shape[1] == 0:
            # Return a vector of large errors, one per split
            results.append([1e6] * splits)
            continue
        X_val_sel = X_val[:, selected]
        model = MLPRegressor(hidden_layer_sizes=(hidden_size,), max_iter=300, random_state=0)
        model.fit(X_sel, y_train)
        y_pred = model.predict(X_val_sel)
        error = median_relative_error(y_val, y_pred)
        print(f"[Eval] Threshold: {threshold:.3f}, Hidden: {hidden_size}, Features: {len(selected)}, MSE: {error:.4f}")
        # Return a vector of the same error, one per split
        results.append([error] * splits)
    return np.array(results)
trials=50
lb = np.zeros(X_scaled.shape[1])
ub = np.ones(X_scaled.shape[1])
dim = len(lb)
bounds = Bounds(np.zeros(dim), np.ones(dim))
kernel = 1.0 * RBF(length_scale=1.0)
exp_w = 0.1
eps = 1e-6
init_pts = 4


# ...existing code...
for f_cores in [4]:  # different trial counts
    results = []

    splits = 4  # keep partitions constant for simplicity
    vpbo = BO(
        distmod=lambda x_batch, *args: np.mean(mlp_eval(x_batch, *args, splits=splits), axis=1).reshape(-1, 1),  # returns scalar per candidate
        args=(X_scaled, Y_scaled),
        dist_ref = {
            'distrefmod': ref_model,
             'distrefmod1': ref_model,  # ... up to split_num
             'distrefmod2': ref_model,
             'distrefmod3': ref_model,
            'distrefmod4': ref_model
            # ... up to split_num
        },
        ref_args=(),
        dim=len(lb),
        bounds=Bounds(np.zeros(dim), np.ones(dim)),
        descale=lambda x: x,
        kernel=1.0 * RBF(length_scale=1.0),
        exp_w=0.1,
        ub=ub,
        lb=lb,
        system=None
    )
    
    # Reset VPBO internal state
    vpbo.y_vpbst = []
    
    lim_init =np.random.uniform(0, 1, vpbo.dim)
    x_init = np.random.uniform(0, 1, vpbo.dim)

    start = time.time()
    print(f"Running VPBO with {splits} partitions...")
    vpbo.optimizer_vpbo(trials=trials, lim_init=lim_init, split_num=splits, af_cores=4, f_cores=f_cores, x_init=x_init)
    elapsed = time.time() - start
    # Flatten y_vpbst into a 1D array of scalars
    all_vals = np.concatenate([np.ravel(arr) for arr in vpbo.y_vpbst])

    results.append({
        "Partitions": splits,
        "Best_MSE": float(np.min(all_vals)),   # ensure clean scalar
        "Time_s": elapsed,
        "Evals": len(all_vals)                 # count total scalar evaluations
    })


df_results = pd.DataFrame(results)
df_results.to_csv("vpbo_partition_results4_50.csv", index=False)

Running VPBO with 4 partitions...
Variable Partitioned BO run...


Shape of y: (131318, 4)
Shape of y: (131318, 4)
Shape of y: (131318, 4)
Shape of y: (131318, 4)
[Eval] Threshold: 0.610, Hidden: 105, Features: 1, MSE: 0.6610
[Eval] Threshold: 0.610, Hidden: 105, Features: 1, MSE: 0.6161
[Eval] Threshold: 0.660, Hidden: 165, Features: 1, MSE: 0.7059
[Eval] Threshold: 0.610, Hidden: 105, Features: 1, MSE: 1.0388
Shape of y: (131318, 4)
[Eval] Threshold: 0.610, Hidden: 105, Features: 1, MSE: 0.6807
ITERATION COUNT IS AT 1;              TOTAL ELAPSED TIME: 74.4


ValueError: The user-provided objective function must return a scalar value.

In [22]:
from sklearn.inspection import permutation_importance
import pandas as pd

def compute_multi_importance(X_full, Y_full, feature_names=None, target_names=None):
    """
    Compute permutation importance per sustainability impact.
    Returns a DataFrame with feature importances.
    """
    n_targets = Y_full.shape[1]
    importances_dict = {}

    # Train separate MLP for each target (easier for interpretation)
    for j in range(n_targets):
        y_target = Y_full[:, j]
        model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=300, random_state=0)
        model.fit(X_full, y_target)

        perm = permutation_importance(model, X_full, y_target, n_repeats=10, random_state=0)
        importances_dict[target_names[j] if target_names else f"Target_{j}"] = perm.importances_mean

    # Combine into DataFrame
    df_importances = pd.DataFrame(importances_dict, index=feature_names if feature_names else [f"Feature_{i}" for i in range(X_full.shape[1])])
    return df_importances

feature_names = [f"Feature_{i}" for i in range(X_scaled.shape[1])]
target_names = [f"Impact_{i}" for i in range(Y_scaled.shape[1])]
df_importances = compute_multi_importance(X_scaled, Y_scaled, feature_names, target_names)
df_importances.to_csv("feature_importances.csv")

In [82]:
def mlp_eval(x_batch, *args, splits=4):
    """
    Evaluates a batch of hyperparameter candidates for VPBO.
    Each split corresponds to one target variable.

    x_batch: (n_candidates, 2) -> [threshold, hidden_size_scaled]
    args: (X_full, Y_full) where Y_full has shape (n_samples, n_targets)
    returns: (n_candidates, splits) array with one error per target.
    """
    X_full, Y_full = args
    n_targets = Y_full.shape[1]
    assert splits == n_targets, f"Expected splits={n_targets}, got {splits}"

    results = []
    x_batch = np.atleast_2d(x_batch)

    for x in x_batch:
        threshold = x[0]
        hidden_size = int(x[1] * (200 - 10) + 10)
        target_errors = []

        for t in range(n_targets):
            y_t = Y_full[:, t]

            # Feature selection for this target
            X_sel, selected = dcor_filter(X_full, y_t, threshold)
            if X_sel.shape[1] == 0:
                target_errors.append(1e6)
                continue

            # Train simple MLP on whole dataset (no CV)
            model = MLPRegressor(
                hidden_layer_sizes=(hidden_size,),
                max_iter=300,
                random_state=0
            )
            model.fit(X_sel, y_t)

            # Predict and evaluate
            y_pred = model.predict(X_sel[:, selected] if X_sel.ndim > 1 else X_sel.reshape(-1, 1))
            error = mean_relative_error(y_t, y_pred)
            target_errors.append(error)

        results.append(target_errors)

        print(f"[Eval] Thr={threshold:.3f}, Hidden={hidden_size}, "
              f"Features={len(selected)}, Errors={target_errors}")

    return np.array(results)
splits = 4 # 4 targets
vpbo = BO(
    distmod=lambda x_batch, *args: mlp_eval(x_batch, *args, splits=splits),
    args=(X_scaled, Y_scaled),
    dist_ref={'distrefmod': ref_model},
    ref_args=(),
    dim=X_scaled.shape[1],  # or define based on the targets if necessary,
    bounds=Bounds(np.zeros(splits), np.ones(splits)),
    descale=lambda x: x,
    kernel=1.0 * RBF(length_scale=1.0),
    exp_w=0.1,
    ub=np.ones(splits),
    lb=np.zeros(splits),
    system=None
)
lim_init =lim_init =  np.random.uniform(0, 1, vpbo.dim)
x_init = np.random.uniform(0, 1, vpbo.dim)

vpbo.optimizer_vpbo(
    trials=20,
    lim_init=lim_init,
    split_num=splits,  # important: match CV folds
    af_cores=2,
    f_cores=4,
    x_init=x_init
)


Variable Partitioned BO run...
Shape of y: (164148,)
Shape of y: (164148,)


IndexError: tuple index out of range

In [67]:
print(type(vpbo.y_vpbst[0]), vpbo.y_vpbst[0])


AttributeError: 'BO' object has no attribute 'y_vpbst'

In [17]:
# ---- Serial BO ----
bo_serial = BO(
    distmod=None,  # not used for sbo
    args=(X_scaled, Y_scaled),
    dist_ref=None,
    ref_args=(),
    dim=X_scaled.shape[1],
    bounds=Bounds(np.zeros(X_scaled.shape[1]), np.ones(X_scaled.shape[1])),
    descale=lambda x: x,
    kernel=1.0 * RBF(length_scale=1.0),
    exp_w=0.1,
    ub=np.ones(X_scaled.shape[1]),
    lb=np.zeros(X_scaled.shape[1]),
    system=mlp_eval   # ✅ callable function, not string
)

start_sbo = time.time()
bo.optimizer_sbo(trials=30, af_cores=4)
sbo_time = time.time() - start_sbo
sbo_best_mse = np.min(bo_serial.y_sbo)

AttributeError: 'BO' object has no attribute 'optimizer_sbo'

In [ ]:
import pandas as pd

data = {
    "Method": ["GridSearch", "Serial BO", "VPBO"],
    "Best_MSE": [gs_mse, sbo_best_mse, vp_best_mse],
    "Time_s": [gs_time, sbo_time, vp_time],
    "Evaluations": [gs_evals, len(bo_serial.y_sbo), len(bo_vp.y_vp)],
    "Params": [gs_params, None, None]  # Params only for GridSearch here
}

df_results = pd.DataFrame(data)
df_results.to_csv("df_results1.csv")

In [ ]:
from itertools import product
import time

def run_gridsearch(X_scaled, Y_scaled, thresholds, hidden_sizes):
    best_score = float('inf')
    best_params = None
    start = time.time()
    eval_count = 0

    for th, hs in product(thresholds, hidden_sizes):
        X_sel, selected = dcor_filter(X_scaled, Y_scaled, th)
        if X_sel.shape[1] == 0:
            continue
        model = MLPRegressor(hidden_layer_sizes=(hs,), max_iter=300, random_state=0)
        X_train, X_val, y_train, y_val = train_test_split(X_sel, Y_scaled, test_size=0.2)
        model.fit(X_train, y_train)
        mse = mean_squared_error(y_val, model.predict(X_val))
        eval_count += 1

        if mse < best_score:
            best_score = mse
            best_params = (th, hs)

    elapsed = time.time() - start
    return best_score, elapsed, eval_count, best_params

grid_points = int(np.sqrt(30))  # ≈ same total evals
thresholds = np.linspace(0, 1, grid_points)
hidden_sizes_scaled = np.linspace(0, 1, grid_points)
hidden_sizes = (hidden_sizes_scaled * (200 - 10) + 10).astype(int)
gs_mse, gs_time, gs_evals, gs_params = run_gridsearch(X_scaled, Y_scaled, thresholds, hidden_sizes)
print(f"[GridSearch] Best MSE={gs_mse:.4f}, Time={gs_time:.2f}s, Evals={gs_evals}, Params={gs_params}")


[GridSearch] Best MSE=0.0000, Time=1311.81s, Evals=20, Params=(0.0, 105)


In [38]:
# Serial BO

bo_serial =  BO(
    distmod=mlp_eval,
    args=(X_scaled, Y_scaled),  # MULTI-TARGET support
    dist_ref={'distrefmod': ref_model},
    ref_args=(),
    dim=X_scaled.shape[1],
    bounds=Bounds(np.zeros(X_scaled.shape[1]), np.ones(X_scaled.shape[1])),
    descale=lambda x: x,
    kernel=1.0 * RBF(length_scale=1.0),
    exp_w=0.1,
    ub=np.ones(X_scaled.shape[1]),
    lb=np.zeros(X_scaled.shape[1]),
    system='mlp_opt'
)
start_sbo = time.time()
bo_serial.optimizer_sbo(trials=30, af_cores=4)
sbo_time = time.time() - start_sbo
sbo_best_mse = np.min(bo_serial.y_sbo)
print(f"[Serial BO] Best MSE={sbo_best_mse:.4f}, Time={sbo_time:.2f}s, Evals={len(bo_serial.y_sbo)}")

# VPBO
bo_vp = BO(
    ub=np.ones(X_scaled.shape[1]),
    lb=np.zeros(X_scaled.shape[1]),
    dim=X_scaled.shape[1],
    exp_w=0.1,
    kernel=1.0 * RBF(length_scale=1.0),
    bounds=Bounds(np.zeros(X_scaled.shape[1]), np.ones(X_scaled.shape[1])),
    args=(X_scaled, Y_scaled),
    distmod=mlp_eval,
    distmod_args=(X_scaled, Y_scaled),
    dist_ref={'distrefmod': ref_model}
)
start_vp = time.time()
bo_vp.optimizer_vpbo(trials=30, split_num=4, lim_init=np.random.uniform(0,1,bo_vp.dim), f_cores=4)
vp_time = time.time() - start_vp
vp_best_mse = np.min(bo_vp.y_vp)
print(f"[VPBO] Best MSE={vp_best_mse:.4f}, Time={vp_time:.2f}s, Evals={len(bo_vp.y_vp)}")


Vanilla BO run...


TypeError: 'str' object is not callable

In [19]:
import matplotlib.pyplot as plt

methods = ["GridSearch", "Serial BO", "VPBO"]
times = [gs_time, sbo_time, vp_time]
mses = [gs_mse, sbo_best_mse, vp_best_mse]
evals = [gs_evals, len(bo_serial.y_sbo), len(bo_vp.y_vp)]

fig, ax1 = plt.subplots()

ax1.bar(methods, times, color='skyblue')
ax1.set_ylabel('Time (s)', color='blue')
ax2 = ax1.twinx()
ax2.plot(methods, mses, marker='o', color='red')
ax2.set_ylabel('MSE', color='red')
plt.title('Accuracy vs Time')

plt.show()

# Optional: evaluations comparison
plt.bar(methods, evals, color='green')
plt.ylabel('# Function Evaluations')
plt.title('Cost Comparison')
plt.show()


NameError: name 'sbo_time' is not defined

In [20]:
import math
import matplotlib.pyplot as plt

# ---- GridSearch ----
def run_gridsearch_same_space(X_full, y_full, total_evals, hidden_min=10, hidden_max=200):
    """
    Grid search over same space as BO:
    - threshold ∈ [0,1]
    - hidden_size_scaled ∈ [0,1] (mapped to [hidden_min, hidden_max])
    total_evals ≈ VPBO trials
    """
    from itertools import product
    start = time.time()
    
    grid_points = int(math.sqrt(total_evals))
    thresholds = np.linspace(0, 1, grid_points)
    hidden_scaled = np.linspace(0, 1, grid_points)
    hidden_sizes = (hidden_scaled * (hidden_max - hidden_min) + hidden_min).astype(int)
    
    best_score = float('inf')
    best_params = None
    eval_count = 0
    
    for th, hs in product(thresholds, hidden_sizes):
        X_sel, selected = dcor_filter(X_full, y_full, th)
        if X_sel.shape[1] == 0:
            continue
        X_train, X_val, y_train, y_val = train_test_split(X_sel, y_full, test_size=0.2)
        model = MLPRegressor(hidden_layer_sizes=(hs,), max_iter=300, random_state=0)
        model.fit(X_train, y_train)
        mse = mean_squared_error(y_val, model.predict(X_val))
        eval_count += 1
        
        if mse < best_score:
            best_score = mse
            best_params = (th, hs)
    
    elapsed = time.time() - start
    return best_score, elapsed, eval_count, best_params

# ---- Serial BO ----
bo_serial = BO(
    distmod=mlp_eval,
    args=(X_scaled, Y_scaled),
    dist_ref={'distrefmod': ref_model},
    ref_args=(),
    dim=X_scaled.shape[1],
    bounds=Bounds(np.zeros(X_scaled.shape[1]), np.ones(X_scaled.shape[1])),
    descale=lambda x: x,
    kernel=1.0 * RBF(length_scale=1.0),
    exp_w=0.1,
    ub=np.ones(X_scaled.shape[1]),
    lb=np.zeros(X_scaled.shape[1]),
    system='mlp_opt'
)
start_sbo = time.time()
bo_serial.optimizer_sbo(trials=30, af_cores=4)
sbo_time = time.time() - start_sbo
sbo_best_mse = np.min(bo_serial.y_sbo)

# ---- VPBO ----
bo_vp = BO(
    distmod=mlp_eval,
    args=(X_scaled, Y_scaled),
    dist_ref={'distrefmod': ref_model},
    ref_args=(),
    dim=X_scaled.shape[1],
    bounds=Bounds(np.zeros(X_scaled.shape[1]), np.ones(X_scaled.shape[1])),
    descale=lambda x: x,
    kernel=1.0 * RBF(length_scale=1.0),
    exp_w=0.1,
    ub=np.ones(X_scaled.shape[1]),
    lb=np.zeros(X_scaled.shape[1]),
    system='mlp_opt'
)
lim_init = np.random.uniform(0, 1, bo_vp.dim)
x_init = np.random.uniform(0, 1, bo_vp.dim)
start_vp = time.time()
bo_vp.optimizer_vpbo(
    trials=30,
    split_num=4,
    lim_init=lim_init,
    f_cores=4,
    af_cores=2,
    ref_cores=2,
    x_init=x_init
)
vp_time = time.time() - start_vp
vp_best_mse = np.min(bo_vp.y_vp)

# ---- GridSearch Run ----
gs_mse, gs_time, gs_evals, gs_params = run_gridsearch_same_space(
    X_scaled, Y_scaled, total_evals=30
)

# ---- Print summary ----
print(f"[GridSearch] Best MSE={gs_mse:.4f}, Time={gs_time:.2f}s, Evals={gs_evals}, Params={gs_params}")
print(f"[Serial BO] Best MSE={sbo_best_mse:.4f}, Time={sbo_time:.2f}s, Evals={len(bo_serial.y_sbo)}")
print(f"[VPBO]      Best MSE={vp_best_mse:.4f}, Time={vp_time:.2f}s, Evals={len(bo_vp.y_vp)}")

# ---- Plots ----
methods = ["GridSearch", "Serial BO", "VPBO"]
times = [gs_time, sbo_time, vp_time]
mses = [gs_mse, sbo_best_mse, vp_best_mse]
evals = [gs_evals, len(bo_serial.y_sbo), len(bo_vp.y_vp)]

# Accuracy vs Time
fig, ax1 = plt.subplots()
ax1.bar(methods, times, color='skyblue')
ax1.set_ylabel('Time (s)', color='blue')
ax2 = ax1.twinx()
ax2.plot(methods, mses, marker='o', color='red')
ax2.set_ylabel('MSE', color='red')
plt.title('Accuracy vs Time')
plt.tight_layout()
plt.show()

# Cost comparison
plt.bar(methods, evals, color='green')
plt.ylabel('# Function Evaluations')
plt.title('Cost Comparison')
plt.tight_layout()
plt.show()


Vanilla BO run...


TypeError: 'str' object is not callable